In [1]:
import json
import pickle
import numpy as np
from collections import Counter


In [2]:
# Load trained artifacts
with open("global_codon_bwt_model.pkl", "rb") as f:
    global_model = pickle.load(f)

with open("aa_models_with_bwt.pkl", "rb") as f:
    aa_models = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

with open("feature_manifest_with_bwt.json") as f:
    feature_manifest = json.load(f)


In [3]:
CODON_TABLE = {
    "TTT":"F","TTC":"F","TTA":"L","TTG":"L",
    "CTT":"L","CTC":"L","CTA":"L","CTG":"L",
    "ATT":"I","ATC":"I","ATA":"I","ATG":"M",
    "GTT":"V","GTC":"V","GTA":"V","GTG":"V",
    "TCT":"S","TCC":"S","TCA":"S","TCG":"S",
    "CCT":"P","CCC":"P","CCA":"P","CCG":"P",
    "ACT":"T","ACC":"T","ACA":"T","ACG":"T",
    "GCT":"A","GCC":"A","GCA":"A","GCG":"A",
    "TAT":"Y","TAC":"Y","TAA":"*","TAG":"*",
    "CAT":"H","CAC":"H","CAA":"Q","CAG":"Q",
    "AAT":"N","AAC":"N","AAA":"K","AAG":"K",
    "GAT":"D","GAC":"D","GAA":"E","GAG":"E",
    "TGT":"C","TGC":"C","TGA":"*","TGG":"W",
    "CGT":"R","CGC":"R","CGA":"R","CGG":"R",
    "AGT":"S","AGC":"S","AGA":"R","AGG":"R",
    "GGT":"G","GGC":"G","GGA":"G","GGG":"G"
}


In [4]:
def load_first_cds(fasta_path):
    seq = ""
    with open(fasta_path) as f:
        for line in f:
            if line.startswith(">"):
                if seq:
                    break
            else:
                seq += line.strip().upper()
    return seq


In [5]:
original_gene = load_first_cds("cds_from_genomic.fna")
print("Original gene length:", len(original_gene))


Original gene length: 363


In [6]:
def bwt_transform(seq):
    seq += "$"
    rotations = [seq[i:] + seq[:i] for i in range(len(seq))]
    rotations.sort()
    return "".join(row[-1] for row in rotations)

def bwt_features(codon_seq):
    bwt = bwt_transform("".join(codon_seq))
    counts = Counter(bwt)
    total = len(bwt)
    entropy = -sum((c/total)*np.log2(c/total) for c in counts.values())
    return {
        "bwt_entropy": entropy,
        "bwt_total_len": total
    }


In [7]:
def predict_best_codon(codon, codon_context):
    aa = CODON_TABLE.get(codon)
    if aa is None or aa == "*":
        return codon  # do not optimize stop codons

    model = aa_models.get(aa, global_model)

    bwt_feat = bwt_features(codon_context)

    # Build feature vector (minimal)
    feature_vector = []
    for f in feature_manifest["bwt_feature_cols"]:
        feature_vector.append(bwt_feat.get(f, 0))

    X = np.array(feature_vector).reshape(1, -1)
    pred = model.predict(X)[0]
    return label_encoder.inverse_transform([pred])[0]


In [8]:
# Split into codons
def split_codons(seq):
    return [seq[i:i+3] for i in range(0, len(seq)-2, 3)]

codons = split_codons(original_gene)


In [9]:
def predict_best_codon(codon, codon_context):
    aa = CODON_TABLE.get(codon)
    
    # Do not optimize stop/invalid codons
    if aa is None or aa == "*":
        return codon

    # Always use global model (safe)
    model = global_model

    # BWT features
    bwt_feat = bwt_features(codon_context)

    # Build feature vector
    feature_vector = []
    for f in feature_manifest["bwt_feature_cols"]:
        feature_vector.append(bwt_feat.get(f, 0))

    X = np.array(feature_vector).reshape(1, -1)
    pred = model.predict(X)[0]

    return label_encoder.inverse_transform([pred])[0]


In [10]:
from collections import Counter
import pandas as pd

# Count codons
orig_counts = Counter(codons)
opt_counts = Counter(optimized_codons)

# Convert to DataFrame
df_compare = pd.DataFrame({
    "Original_Count": orig_counts,
    "Optimized_Count": opt_counts
}).fillna(0).astype(int)

# Add difference column
df_compare["Difference"] = df_compare["Optimized_Count"] - df_compare["Original_Count"]

# Sort by most changed codons
df_compare = df_compare.sort_values("Difference", ascending=False)

df_compare.head(15)


NameError: name 'optimized_codons' is not defined

In [ ]:
# Show codons that increased the most
df_compare.sort_values("Difference", ascending=False).head(15)


,Original_Count,Optimized_Count,Difference
TCA,1,0,-1
AAA,1,0,-1
TAG,1,0,-1
TGG,1,0,-1
ATT,1,0,-1
AGC,1,0,-1
AGT,1,0,-1
GAT,1,0,-1
GTG,2,0,-2
CAC,2,0,-2


In [11]:
def protein_from_dna(seq):
    protein = ""
    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        protein += CODON_TABLE.get(codon, "?")
    return protein


In [12]:
def build_full_feature_vector(codon_context):
    """
    Builds a feature vector with EXACTLY the same length and order
    as used during training.
    Missing features are filled with 0.
    """
    # Compute BWT features
    bwt_feat = bwt_features(codon_context)

    # Initialize full feature dict
    full_features = {}

    # Fill known BWT features
    for k, v in bwt_feat.items():
        full_features[k] = v

    # Build vector in training order
    feature_vector = []
    for feature_name in feature_manifest["all_feature_cols"]:
        feature_vector.append(full_features.get(feature_name, 0))

    return np.array(feature_vector).reshape(1, -1)


In [13]:
def predict_best_codon(codon, codon_context):
    aa = CODON_TABLE.get(codon)

    # Do not optimize stop or invalid codons
    if aa is None or aa == "*":
        return codon

    # Always use global model (safe)
    model = global_model

    # Build correct feature vector
    X = build_full_feature_vector(codon_context)

    # Predict
    pred = model.predict(X)[0]
    return label_encoder.inverse_transform([pred])[0]


In [14]:
import pandas as pd

codon_stats = pd.read_csv("per_AA_stats_with_bwt.csv")
print(codon_stats.columns)


Index(['aa', 'rows', 'codon_options', 'test_rows', 'accuracy', 'weight',
       'weighted_acc'],
      dtype='object')


In [15]:
import pandas as pd
import math
from collections import defaultdict

# Load codon usage safely
codon_usage_df = pd.read_csv("codon_usage.csv", low_memory=False)

# Normalize column names
codon_usage_df.columns = [c.upper() for c in codon_usage_df.columns]

# Detect codon columns (AAA, AAG, ...)
codon_cols = [c for c in codon_usage_df.columns if len(c) == 3]

# Pick FIRST species row (or change species ID explicitly)
species_row = codon_usage_df.iloc[0]

# Build codon frequency dict (DNA format)
codon_freq = {}
for codon in codon_cols:
    dna_codon = codon.replace("U", "T")
    try:
        codon_freq[dna_codon] = float(species_row[codon])
    except:
        codon_freq[dna_codon] = 0.0

# Build AA → codons mapping
aa_to_codons = defaultdict(list)
for codon, aa in CODON_TABLE.items():
    if aa != "*":
        aa_to_codons[aa].append(codon)

# Compute CAI weights
cai_weights = {}
for aa, codons_list in aa_to_codons.items():
    freqs = [codon_freq.get(c, 0) for c in codons_list]
    max_freq = max(freqs) if max(freqs) > 0 else 1.0
    for c in codons_list:
        cai_weights[c] = codon_freq.get(c, 0) / max_freq

# Safe CAI computation
def compute_cai(codons, weights):
    valid_weights = [weights[c] for c in codons if c in weights and weights[c] > 0]
    if len(valid_weights) == 0:
        return 0.0
    return math.exp(sum(math.log(w) for w in valid_weights) / len(valid_weights))

cai_original = compute_cai(codons, cai_weights)
cai_optimized = compute_cai(optimized_codons, cai_weights)

print("Original CAI:", round(cai_original, 4))
print("Optimized CAI:", round(cai_optimized, 4))


NameError: name 'optimized_codons' is not defined

In [16]:
def choose_best_codon(top_codons, cai_weights):
    best = None
    best_score = -1

    for codon, prob in top_codons:
        cai = cai_weights.get(codon, 0)
        score = 0.7 * prob + 0.3 * cai   # ML + CAI

        if score > best_score:
            best_score = score
            best = codon

    return best


In [17]:
def predict_top_k_codons(codon, context, k=3):
    aa = CODON_TABLE.get(codon)
    if aa is None or aa == "*":
        return [(codon, 1.0)]

    # Choose correct model
    model = aa_models.get(aa, global_model)

    # Build features
    bwt_feat = bwt_features(context)
    X = build_feature_vector(codon, bwt_feat)

    # Predict probabilities
    probs = model.predict_proba(X)[0]

    # Decode codons
    codon_list = label_encoder.classes_

    top_indices = probs.argsort()[-k:][::-1]
    return [(codon_list[i], probs[i]) for i in top_indices]
